# 回帰(Regression)と分類(Classification)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kyorin-phys/MLIntro/blob/main/3/3.ipynb)

<img src="linear-vs-logistic.png" width="75%">


image from [ML for Beginners](https://github.com/microsoft/ML-For-Beginners/tree/main/2-Regression/4-Logistic)

* 目的関数の違い：　
スケール変数とカテゴリー変数

* 線形回帰：
1変数、多変数

* ロジスティック回帰(**回帰**という名前だが実際は**分類**が目的)：

    * 二値（０か１か）： sigmoid関数

    * 多クラス（3種類以上のどれに近いか）: softmax関数、one-hot表現  

数値データなら深層学習でなくてもできる

アヤメデータを使う

* 特徴量：がく片、花びらの幅と長さ　４変数
* 目的変数：花の種類

<img src="https://miro.medium.com/max/1000/1*Hh53mOF4Xy4eORjLilKOwA.png" width="50%">

## seaborn ライブラリ　
統計用グラフが充実
* [seaborn](https://seaborn.pydata.org/index.html)

[グラフ一覧](https://seaborn.pydata.org/examples/index.html)

[matplotlib & seaborn](https://www.amazon.co.jp/dp/4798055433) これだけで本になるくらいなので使いこなすのは大変だが、そのまま出版に使えるような品質の高いグラフが作れる。関数名で検索すれば、用例を解説したページが多数ある。

In [ ]:
# アヤメデータ(前回はCSVファイルを使ったが、データセットがライブラリに用意されている)
import seaborn as sns # なぜSNSと略すのかはアメリカのテレビ番組が由来なので説明されてもわからない
# from sklearn.datasets import load_iris # sklearn からデータを読むこともできる
print(sns.get_dataset_names()) # seabornに含まれるサンプルデータ
iris = sns.load_dataset('iris')
# このデータはpandasのDataFrame形式になっている
iris.head() 

In [ ]:
# sepal_length, sepal_widthで散布図、hue で層別に色分け(済)
sns.relplot(data=iris, x='petal_length',y='petal_width', hue='species')

各特徴量のペア毎の散布図をまとめて描いたり、相関係数を視覚的に確認できる。

In [ ]:
# 散布図行列　
sns.pairplot(iris, hue='species', corner=True) # 対角線をまたいで対称なので下半分だけ表示

In [ ]:
# 相関行列
corr = iris.corr(numeric_only=True) #カテゴリー変数speciesを除く
corr

In [ ]:
# 相関行列のヒートマップ
sns.heatmap(corr, cmap='coolwarm',annot=True)

## [scikit-learn](https://scikit-learn.org/stable/index.html) (sklearn)

データ分析に有用なライブラリ
* LinearRegression: 線形回帰
* LogisticRegression: ロジスティック回帰
* train_test_split: 訓練用と検証用にデータを分割する

supervised learning(教師あり学習): 特徴量と目的変数のセットが必要

<img src='split.png' width='50%'>

訓練データで回帰式を求め、検証データでどれくらい再現できているかを検証する

* root_mean_squared_error, r2_score: 
結果の良し悪しを判断 (平均二乗平方根誤差、相関(決定係数))

* 線形回帰、ロジスティック回帰、その他の方法が全て同じ構文で統一されている。
```
model = モデル名() # モデルを指定
model.fit() # 学習
model.predict() # 予測
```


In [ ]:
# 準備
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, root_mean_squared_error

# アヤメデータセットを読み込み
iris = sns.load_dataset('iris') #DataFrame形式で読み込み　
iris.head()

In [ ]:
# ファイルに書き出しできる
#iris.to_csv('iris.csv', index=False) # 1列目にインデックスを追加しない
#iris.to_excel('iris.xlsx', index=False)
#df = pd.read_csv('iris.csv') # 読み込み
#df = pd.read_excel('iris.xlsx')
#df.head()

In [ ]:
# setosa だけ抽出したいとき
setosa = iris[iris['species']=='setosa']
setosa.tail()

In [ ]:
# がく片の幅と長さのデータを抽出
# 単回帰
X = iris[['petal_length']] # ベクトルではなく、行列形式
y = iris['petal_width']    # ベクトル

# 訓練データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
print(X_train.size, X_test.size, len(X)*0.3) # Xのサイズにtest_sizeを掛けた数が検証用データの個数

# 線形回帰モデルを作成
model = LinearRegression()

# モデルを訓練
model.fit(X_train, y_train)

# テストデータで予測
y_pred = model.predict(X_test)

print(list(y_pred))
print(list(y_test))

print('傾き:',model.coef_[0], 'y切片', model.intercept_)

In [ ]:
# モデルの評価
rmse = root_mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Root Mean Squared Error: {rmse}')
print(f'R^2 Score: {r2}')

# 回帰直線をプロット
#plt.figure(figsize=(10, 6))
plt.scatter(X_test, y_test, color='blue', label='Actual data')
plt.plot(X_test, y_pred, color='red', linewidth=2, label='Regression line')
plt.xlabel('Petal Length (cm)')
plt.ylabel('Petal Width (cm)')
plt.title('Linear Regression: Petal Length vs Petal Width')
plt.legend()
plt.show()

## ロジスティック回帰

* 目的変数が**2値**の分類（真偽）
* **多クラス分類**（アヤメでどの種に属するか）
各クラスの確率を求めて、最も高い確率のクラスに属すると予測

`sklearn.model_selection.train_test_split` データを訓練用(train)、検証用(test)に分割する。

ロジスティック回帰の性能評価（正しく予想できるかを定量的に判断する）

`sklearn.metrics.accuracy_score` 正解率

`sklearn.metrics.classification_report`　分類レポート

### 2値の例
アヤメの種がsetosaであるかどうかをロジスティック回帰で予測する

In [ ]:
# 準備
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression # ロジスティック回帰
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score # 2値分類なので平均二乗誤差を使わない

# アヤメデータセットを読み込み
iris = load_iris()
X = iris.data
y = iris.target
y
# y=0: setosa, y=1: versicolor, y=2: virginica
# それぞれの種が50個ずつ

In [ ]:
# setosa なら1、その他は0 
y_bin = (y==0).astype(int) # bool型numpy配列をまとめてint型に変換
y_bin

In [ ]:
X.size, y_bin.size # Xは4変数x150件, yは150件

In [ ]:
# setosa かどうかを2値分類する
# 訓練データとテストデータに分割（順序をシャッフルし、乱数を固定するためにrandom_stateを指定）
X_train, X_test, y_train, y_test = train_test_split(X, y_bin, test_size=0.2, random_state=0)

# ロジスティック回帰モデルを作成
model = LogisticRegression()

# モデルを訓練
model.fit(X_train, y_train)

# テストデータで予測
y_pred = model.predict(X_test)

print('予測', list(y_pred))
print('正解', list(y_test))

print('正解率', accuracy_score(y_test, y_pred))

In [ ]:
150*0.2, len(y_pred) # 150件のうち20%を検証用に使った

`test_size`, `random_state`の値を変えると結果が変わるかどうか試してみよ。

test_sizeを大きくすると学習データが減るので、正解率は下がるはずだが、setosaは孤立しているので、ほとんど変わらない。

random_stateは乱数の出方を変えるだけなので、データの選択によって偶然変わることがあるが、本質的には変わらないはず。

setosa以外の種について同様にやってみよ。

## 多クラス分類
アヤメがどの種に属するかを予測する

In [ ]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix,ConfusionMatrixDisplay
from sklearn.multiclass import OneVsRestClassifier # 多クラス分類用

# Irisデータセットをロード
iris = load_iris()
X = iris.data
y = iris.target

# データをトレーニングセットとテストセットに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# OneVsRestClassifierを使用して多クラス分類のロジスティック回帰モデルを作成してトレーニング
model = OneVsRestClassifier(LogisticRegression(max_iter=100))
model.fit(X_train, y_train)

# テストセットで予測を行う
y_pred = model.predict(X_test)

# モデルの精度を計算
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=iris.target_names)
cm = confusion_matrix(y_test, y_pred)

print(f"アヤメの種類を予測するロジスティック回帰モデルの精度: {accuracy:.2f}")
print("分類レポート:")
print(report)

print('混同行列')
print(cm)

# Plot the confusion matrix
ConfusionMatrixDisplay.from_estimator(model, X_test, y_test, display_labels=iris.target_names)
plt.title('Confusion Matrix')
plt.show()

### confusion matrix(混同行列)


| 実際　\　予測    | 陽性 | 陰性 |
|----|----|----|
|陽性 | TP   | FN   |
|陰性　| FP | TN |

* 正解率：全体の中で正しく分類できた割合
  
$$\mathrm{accuracy = \frac{TP+TN}{TP+TN+FP+FN}}$$

* 適合率（精度）：陽性と予測された中で実際に陽性だった割合 (陽性的中率)

$$\mathrm{precision = \frac{TP}{TP+FP}}$$

* 再現率：実際に陽性であるデータの中で正しく陽性と予測できた割合 (感度sensitivity)

$$\mathrm{recall = \frac{TP}{TP+FN}}$$

* F1スコア： 適合率と再現率の調和平均（バランスを取る。どちらかが極端に高い・低い場合に有用）

$$\mathrm{F1score = \frac{2\times precision \times recall}{precision + recall}}$$

# one-hot 表現
機械学習（深層学習）ではカテゴリー変数に対して
1 = [1,0,0]
2 = [0,1,0]
3 = [0,0,1]
のような変換を行う。
手書き数字の分類（0~9）は10列の[0,1]のデータを作る。
統計ソフトではダミー変数という。

# その他のデータセット
sklearn.datasets にはいくつかのサンプルデータが用意されている。

* [diabetes](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_diabetes.html)　糖尿病患者　回帰用
* [breast cancer](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html) 乳癌診断結果　分類用

In [ ]:
from sklearn.datasets import load_diabetes
from sklearn.datasets import load_breast_cancer
import pandas as pd

diabetes = load_diabetes()
print(diabetes.DESCR)

df = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)
df['target'] = diabetes.target
df.head()

In [ ]:
cancer = load_breast_cancer()
print(cancer.DESCR)
df = pd.DataFrame(cancer.data, columns=cancer.feature_names)
df['target'] = cancer.target
df.head()

# 統計用ライブラリ
* [statsmodels](https://www.statsmodels.org/stable/index.html) Rと似た形式にまとめられている。　
* [scipy.stats](https://docs.scipy.org/doc/scipy/reference/stats.html) `linregress`という線形回帰用の関数があるが、単回帰しか扱えない。
(`sklearn.linear_model.LinearRegression`は重回帰と単回帰を同じ方法で行う。)
統計検定用の関数が各種用意されている。

これらのライブラリは統計学的解析に使われる。情報が多過ぎるのでここでは紹介するにとどめる。